<a href="https://colab.research.google.com/github/AbimaelLopez/Diplomado/blob/main/10_SVM_ejercicio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **10. Support Vector Machine**

### Objetivos
Que la o el estudiante aprenda a utilizar el algoritmo Support Vector Machine dentro de la biblioteca Sklearn.

### Descripción del procedimiento a seguir



### Datos
**Profesora**: Dra. Jessica Beltrán Márquez<br>
Maestría en Ciencia de Datos y Optimización<br>
Centro de Investigación en Matemáticas Aplicadas <br>
Universidad Autónoma de Coahuila


### Bibliografía
1. http://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html
2. https://raw.githubusercontent.com/JoaquinAmatRodrigo/Estadistica-machine-learning-python/master/data/Boston.csv



## **1. Importar bibliotecas**

In [2]:
#Importamos las bibliotecas que vamos a utilizar
import math
import random
import pandas as pd
import numpy as np

#Datasets
from sklearn.datasets import load_iris

#Model selection
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold, KFold

#Metrics
from sklearn.metrics import accuracy_score

#Classfier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

#Preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

#Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import ColumnTransformer

## **2. Uso de SVM para clasificar el conjunto de datos Iris**

### **2.1 Carga y separación del conjunto de datos Iris**
- Leemos los datos
- Separamos en X y y
- Separamos en datos de entrenamiento y de prueba

In [3]:
# Nombre del archivo csv con el conjunto de datos
#dataset_filename = '/content/drive/MyDrive/Curso ML Diplomado/10. SVM/irisDataset.csv'
# Cargar los datos a un dataframe pandas
#df = pd.read_csv(dataset_filename)
#df.head()

iris = load_iris()


# Create a Pandas DataFrame
X = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# Add the target variable (species) to the DataFrame
y = pd.Series(iris.target)


In [4]:
#X = df.iloc[:,:-1]
#y= df.iloc[:,-1]

X_train,X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=10, stratify=y)

#Obtener las clases de flores a partir de la última columna
classes = y.unique()
print(classes)

[0 1 2]


### **2.2 Instanciar un clasificador SVM lineal**


In [5]:
#Create a SVM classifier
clf = svm.SVC(kernel='linear')

### **2.3 Ajustar el clasificador a los datos**

In [6]:
clf.fit(X_train,y_train)

SVC(kernel='linear')

### **2.4 Predecir los targets del conjunto de prueba**

In [7]:
y_pred = clf.predict(X_test)

### **2.5 Obtener la exactitud de la clasificación en el conjunto de prueba**

In [8]:
test_accuracy = accuracy_score(y_test,y_pred)
print(test_accuracy)

1.0


## **3. Comparar varios algoritmos de clasificación sobre el Titanic dataset**

- Se comparan los modelos:
  - Naive Bayes
  - KNN
  - Árboles de decisión
  - SVM

Asimismo, en esta sección se definen Pipelines y se prueban hiperparámetros para cada modelo.

### **3.1 Carga y separación de datos**

In [9]:
# Load the Titanic dataset (replace the path with your actual file path)
titanic_data = pd.read_csv('/content/titanic (1).csv')

# Drop irrelevant columns
titanic_data = titanic_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

# Separate features and target variable
X = titanic_data.drop('Survived', axis=1)
y = titanic_data['Survived']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)



### **3.2 Definición de Pipeline para preprocesamiento de datos numéricos y categóricos**

In [10]:
# Define numerical and categorical features
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Create preprocessing steps for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Create column transformer to apply transformations to numerical and categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])


### **3.3 Instanciamiento de los 4 clasificadores**

In [11]:
# each of these models will take a turn as the second Pipeline step
clf1 = CategoricalNB()
clf2 = KNeighborsClassifier()
clf3 = DecisionTreeClassifier()
clf4 = svm.SVC()

### **3.4 Creación de Pipeline con preprocesamiento y clasificador**
NOTA: Se deja un clasificador como *placeholder*, pero esto se sobreescribe con los parámetros definindos.

In [12]:
# create the Pipeline
pipe = Pipeline([('preprocessor', preprocessor), ('classifier', clf1)])

### **3.5 Se definen diccionarios de hiperparámetros para cada modelo**

In [13]:
# create the parameter dictionary for clf1
params1 = {}
params1['classifier__alpha'] = [1.0, 2.0]
params1['classifier'] = [clf1]

# create the parameter dictionary for clf2
params2 = {}
params2['classifier__n_neighbors'] =  [3, 5, 7]
params2['classifier__weights'] = ['uniform', 'distance']
params2['classifier'] = [clf2]

# create the parameter dictionary for clf2
params3 = {}
params3['classifier__criterion'] = ['gini', 'entropy']
params3['classifier__max_depth'] = [None, 5, 10, 15]
params3['classifier'] = [clf3]

params4 = {}
params4['classifier__kernel']= ['linear', 'rbf', 'poly']
params4['classifier__C'] = [0.1,1,10]
params4['classifier__gamma']= ['scale', 'auto']
params4['classifier__degree']= [2,3,4]
params4['classifier'] = [clf4]

### **3.6 Creación de lista de diccionarios de parámetros**

In [14]:
# create a list of parameter dictionaries
params = [params1, params2, params3, params4]

### **3.7 Busqueda de mejores modelos con *GridSearchCV***

In [15]:
# this will search every parameter combination within each dictionary
grid = GridSearchCV(pipe, params)
grid.fit(X_train, y_train)
grid.best_params_
print(grid.best_params_)

{'classifier': SVC(C=10, degree=2), 'classifier__C': 10, 'classifier__degree': 2, 'classifier__gamma': 'scale', 'classifier__kernel': 'rbf'}


### **3.8 Acceso al mejor modelo y evaluación en conjunto de prueba**

In [16]:
# Access the best model (best estimator) directly
best_model = grid.best_estimator_
best_model_name = best_model.named_steps['classifier'].__class__.__name__
# Now you can use the best_model for further analysis or evaluation
best_predictions = best_model.predict(X_test)
best_accuracy = accuracy_score(y_test, best_predictions)

print(f"Best Model Test Accuracy: {best_accuracy}")

print(f"Best Model: {best_model_name}")
print(f"Best Model parameters: {best_model['classifier'].get_params()}")


Best Model Test Accuracy: 0.7932960893854749
Best Model: SVC
Best Model parameters: {'C': 10, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}


## **4. Ejercicio: Regresión con SVM**

Usar SVM para regresión para predecir el valor incluido en 'MEDV' en el conjunto de datos Boston house.
NOTA: Ya tienes la mayoría de estos pasos en la práctica de Trees and Overfitting.

Pasos
- Leer los datos y separarlos correctamente
- Definir un pipeline haciendo uso de ColumnTransformer. Identifica las columnas numéricas y define un imputador con estrategia de media y un escalador Standard.
- Importar la clase correspondiente a árboles de decisión para regresión
- Hacer los cambios necesarios para trabajar con problemas de regresión, incluyendo no usar estratificación en la separación de datos y revisar las métricas.
- Define un espacio de búsqueda de hiperparámetros de la SVM para regresión. Kernels: linear y rbf, valor C: 0.1,1,10,100
- Haz la búsqueda de hiperparámetros con GridSearchCV. Usa la métrica MSE.
- Evalúa en el conjunto de prueba.


- http://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html
- https://raw.githubusercontent.com/JoaquinAmatRodrigo/Estadistica-machine-learning-python/master/data/Boston.csv

In [17]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
#Metrics
from sklearn.metrics import r2_score
from sklearn import svm

In [29]:
url = "https://raw.githubusercontent.com/JoaquinAmatRodrigo/Estadistica-machine-learning-python/master/data/Boston.csv"
#column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS','NOX', 'RM', 'AGE', 'DIS', 'RAD', 'RAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
#boston_data = pd.read_csv(url, names=column_names)

boston_data = pd.read_csv(url)

boston_data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2


In [30]:
# Select features and target variable
X = boston_data[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'LSTAT']]
y = boston_data['MEDV']
X.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,12.653063
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,7.141062
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,1.730000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,6.950000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,11.360000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,16.955000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,37.970000


In [31]:
rows_with_nan = X[X.isna().any(axis=1)]
print("Rows with NaN values:\n", rows_with_nan)

Rows with NaN values:
 Empty DataFrame
Columns: [CRIM, ZN, INDUS, CHAS, NOX, RM, AGE, DIS, RAD, TAX, PTRATIO, LSTAT]
Index: []


In [32]:
# Split the data into training and testing sets using stratified split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
# Define numerical and categorical features
numerical_features = X.select_dtypes(include=['float64']).columns
categorical_features = X.select_dtypes(include=['int64']).columns

# Create preprocessing steps for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Create column transformer to apply transformations to numerical and categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [34]:
# each of these models will take a turn as the second Pipeline step
rg1 = DecisionTreeRegressor()
rg2 = svm.SVR()

In [35]:
# create the Pipeline
pipe = Pipeline([('preprocessor', preprocessor), ('regressor', rg1)])

In [39]:
# create the parameter dictionary for rg1
params1 = {}
#params1['classifier__alpha'] = [1.0, 2.0]
params1['regressor__criterion'] = ['squared_error','absolute_error','friedman_mse', 'poisson']
params1['regressor__max_depth'] = [None, 5, 10, 15, 20]
params1['regressor__min_samples_split'] = [2, 5, 10]
params1['regressor__min_samples_leaf']= [ 1, 2, 4,8]
params1['regressor'] = [rg1]

# create the parameter dictionary for rg2
params2 = {}
params2['regressor__kernel']= ['linear', 'rbf', 'poly']
params2['regressor__C'] = [0.1,1,10]
params2['regressor__gamma']= ['scale', 'auto']
params2['regressor__degree']= [2,3,4]
params2['regressor'] = [rg2]

# create the parameter dictionary for clf2
#params2 = {}
#params2['regressor__n_neighbors'] =  [3, 5, 7]
#params2['regressor__weights'] = ['uniform', 'distance']
#params2['regressor'] = [rg2]

In [40]:
# create a list of parameter dictionaries
params = [params1, params2]

In [41]:
# this will search every parameter combination within each dictionary
grid = GridSearchCV(pipe, params,scoring='neg_mean_squared_error', error_score='raise')
grid.fit(X_train, y_train)
grid.best_params_
print(grid.best_params_)

{'regressor': SVR(C=10, degree=2), 'regressor__C': 10, 'regressor__degree': 2, 'regressor__gamma': 'scale', 'regressor__kernel': 'rbf'}


In [42]:
# Access the best model (best estimator) directly
best_model = grid.best_estimator_
best_model_name = best_model.named_steps['regressor'].__class__.__name__
# Now you can use the best_model for further analysis or evaluation
best_predictions = best_model.predict(X_test)
best_accuracy = r2_score(y_test, best_predictions)

print(f"Best Model Test Accuracy: {best_accuracy}")

print(f"Best Model: {best_model_name}")
print(f"Best Model parameters: {best_model['regressor'].get_params()}")

Best Model Test Accuracy: 0.8138784249450305
Best Model: SVR
Best Model parameters: {'C': 10, 'cache_size': 200, 'coef0': 0.0, 'degree': 2, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': False}
